## Chapter 14 - Functional Tools for Nested Data

Let's start by creating a function that modifies a field in a struct.
Struct are immutable, unless we use the mutable flag. Using mutable structs
can be less efficient, so we instead will use the conventional struct with
the `Accessors.jl` package to more easily "update them". Note the quotes in "update".
The reason is that we don't actually update immutable structs instances, we just
copy them with a different value in one field.

In [1]:
using Pkg
Pkg.activate(".")
using Accessors

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/FunctionalProgramming`
┌ Info: Precompiling Accessors [7d9f7c33-5ae7-4f3b-8dc6-eff91059b697]
└ @ Base loading.jl:1664


In [2]:
struct MyData
    name::String
    size::Int
    MyData(name,size) = size ≥ 0 ? new(name,size) : error("Size must be ≥ 0")
end

In [3]:
item = MyData("Davi",170)

MyData("Davi", 170)

In [4]:
# function setfield(item, field::Symbol, value)
#     ex = quote
#         @set $item.$field = $value
#     end
#     return eval(ex)
# end
setfield(item,field, value) = set(item, PropertyLens(field), value)

setfield (generic function with 1 method)

Note that our `setfield` function is the same as the `objectSet` function from the book.
Now we can define the `update` function.

In [6]:
function update(item, key, modify::Function)
    value = getfield(item, key)
    newitem = setfield(item, key, modify(value))
    return newitem
end

update (generic function with 1 method)

In [7]:
update(item,:name,x->"Ok")

MyData("Ok", 170)

Note that the function returns a copy of the item with the modification, hence,
it can be seen as a pure function, i.e. not side-effects, and hence, we could
consider it to be a calculation and not an action.

## Using update()

In [8]:
struct Employee
    name::String
    salary::Real
    Employee(name,salary) = salary ≥ 0 ? new(name,salary) : error("Salary must be ≥ 0")
end

In [9]:
employee = Employee("Kim", 120_000)

function raise10perc(salary::Real)
    return salary*1.1
end

update(employee,:salary,raise10perc)

Employee("Kim", 132000.0)

## Applying Update To Nested Data

Our code works fine with "shallow" structs. But what if it was more like a json object, with various nestings?

In [10]:
struct Options
    color::String
    size::Int
end
struct Clothing
    name::String
    price::Real
    options::Options
end

Clothing(name,price,color,size) = Clothing(name,price,Options(color,size));
shirt = Clothing("shirt", 13, "blue", 3)

Clothing("shirt", 13, Options("blue", 3))

How can we update the option size? 

In [11]:
update(shirt,:options, x-> update(x,:size,x->2))

Clothing("shirt", 13, Options("blue", 2))

Let's write this in the form of a function. 

In [12]:
update2(item, key1, key2, modify) = update(item,key1, x-> update(x,key2,modify))

update2(shirt, :options,:size,x->2)

Clothing("shirt", 13, Options("blue", 2))

It' works beautifully. But we can see that there is the possibility of writing a more
generict `update` function. One that can go as deep as the number of keys passed.

In [13]:
function updatenested(item,keys::Array, modify::Function)
    if length(keys) === 0
        return modify(item)
    end
    restkeys = copy(keys)
    key = popfirst!(restkeys)
    return update(item, key, x->updatenested(x, restkeys,modify))
end

keys = [:options,:size]
updatenested(shirt, keys, x -> 2)

Clothing("shirt", 13, Options("blue", 2))

Let's test it in a more nested object. 

In [14]:
struct Cart
    products::Clothing
end

In [15]:
cart = Cart(shirt)

updatenested(cart, [:products,:options,:size], x -> 2)

Cart(Clothing("shirt", 13, Options("blue", 2)))

## Deeply Nested Data Struct 

In [9]:
struct Mark
    name
    child::Union{Mark,Nothing}
end

In [11]:
Mark("ok",nothing)

Mark("ok", nothing)